In [1]:
import os
os.environ['PJRT_DEVICE'] = 'TPU'


In [2]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
import csv
for dirname, _, filenames in os.walk('/kaggle/input'):
    print(dirname)

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input


/kaggle/input/mynumpyfiles


/kaggle/input/mynumpyfiles/NumpyDS


/kaggle/input/mynumpyfiles/NumpyDS/test_Npy_fold_1


/kaggle/input/mynumpyfiles/NumpyDS/train_Npy_fold_4


...

/kaggle/input/mynumpyfiles/NumpyDS/test_Npy_fold_2


/kaggle/input/mynumpyfiles/NumpyDS/train_Npy_fold_0


/kaggle/input/mynumpyfiles/NumpyDS/test_Npy_fold_4


/kaggle/input/mynumpyfiles/NumpyDS/test_Npy_fold_3


/kaggle/input/mynumpyfiles/NumpyDS/train_Npy_fold_3


In [3]:
import torch_xla
import torch_xla.debug.metrics as met
import torch_xla.distributed.parallel_loader as pl
import torch_xla.utils.utils as xu
import torch_xla.core.xla_model as xm
import torch_xla.distributed.xla_multiprocessing as xmp
import torch_xla.test.test_utils as test_utils

from torch_xla.distributed.parallel_loader import MpDeviceLoader

import warnings
warnings.filterwarnings("ignore")

/usr/local/lib/python3.10/site-packages/torch_xla/__init__.py:253: UserWarning: `tensorflow` can conflict with `torch-xla`. Prefer `tensorflow-cpu` when using PyTorch/XLA. To silence this warning, `pip uninstall -y tensorflow && pip install tensorflow-cpu`. If you are in a notebook environment such as Colab or Kaggle, restart your notebook runtime afterwards.
  warnings.warn(


In [4]:
myMeta="/kaggle/input/mynumpyfiles/NumpyDS/GlobalFoldsFiles_Train_Test_Augmented_Cleaned.csv"
myGlobalDs=pd.read_csv(myMeta)
myGlobalDs.drop(myGlobalDs.columns[myGlobalDs.columns.str.contains('unnamed',case = False)],axis = 1, inplace = True)
myGlobalDs["FilePath"] = [fpath.replace("\\", "/") for fpath in myGlobalDs["FilePath"]]

In [5]:
myGlobalDs.head()

,Fold,Type,nb,Label,condLabel,FileName,FilePath,LabelIDs
0,0,Test,5.0,10_T02 M06 (VHM 309-12),VHM,0006_label_T02 M06 (VHM 309-12).npy,test_Npy_fold_0/0006_label_T02 M06 (VHM 309-12...,0
1,0,Test,21.0,10_T02 M06 (VHM 309-12),VHM,0022_label_T02 M06 (VHM 309-12).npy,test_Npy_fold_0/0022_label_T02 M06 (VHM 309-12...,0
2,0,Test,27.0,10_T02 M06 (VHM 309-12),VHM,0028_label_T02 M06 (VHM 309-12).npy,test_Npy_fold_0/0028_label_T02 M06 (VHM 309-12...,0
3,0,Test,29.0,10_T02 M06 (VHM 309-12),VHM,0030_label_T02 M06 (VHM 309-12).npy,test_Npy_fold_0/0030_label_T02 M06 (VHM 309-12...,0
4,0,Test,33.0,10_T02 M06 (VHM 309-12),VHM,0034_label_T02 M06 (VHM 309-12).npy,test_Npy_fold_0/0034_label_T02 M06 (VHM 309-12...,0


In [6]:
myGlobalDs.describe()

,Fold,nb,LabelIDs
count,11780.000000,7215.000000,11780.000000
mean,2.000000,991.312266,4.573005
std,1.414274,533.009652,3.262599
min,0.000000,5.000000,0.000000
25%,1.000000,515.000000,2.000000
50%,2.000000,1010.000000,5.000000
75%,3.000000,1477.000000,7.000000
max,4.000000,1837.000000,10.000000


In [7]:
import torch
import torchaudio
from torchaudio import transforms

import math, random


import matplotlib.pyplot as plt


class DataUtils():
    #---------------------------------------------
    #Load a file and returns it as a list
    #---------------------------------------------
    @staticmethod
    def x_loading(subsetXTrainFiles, myDir):
        #Takes a list of files and returns a numpy array of shape (nObservations, nChannels, nSamples)
        X_subset=[]
        
        for file in subsetXTrainFiles:    
            myArray=np.load(os.path.join(myDir, file))
            X_subset.append(myArray)
        X_subset=np.array(X_subset) #X_subset.shape (1989 observations, 4 channels :sound, accel X,Y,Z, 250062 samples per observation)
        
        return X_subset
    
    @staticmethod
    def saveFile(myNpy, destDir, fileName):
        """Save a file."""
        np.save(os.path.join(destDir, fileName), myNpy)

    #---------------------------------------------
    #Align the data to have a fixed duration.
    #---------------------------------------------
    @staticmethod
    def padTruncate(mySig, maxTime, fs):
        """Pad or truncate the data to have a fixed duration.
        Args:
            mySig: the signal as a list
            maxTime: The maximum time in milliseconds.
            fs: the sampling rate in Hz    
            """
        sLen = mySig.shape[0]
        trunc=0
        maxLen=int(maxTime*int(fs/1000))
        if sLen > maxLen:
            trunc=1
            mySig = mySig[:, :maxLen]
        elif sLen < maxLen:
            trunc=-1
            mySig = np.pad(mySig, ((0, int(maxLen - sLen)),(0,0)), mode='wrap')
        
        return (trunc, len(mySig), mySig)
    
    def addGaussianNoise(mySig, std=0.02):
        """Add a gaussian noise."""
        mySig_float=mySig.astype(np.float32)
        noise = np.random.normal(0, std*np.std(np.abs(mySig_float), axis=0), mySig.shape)
        noisySig=np.clip(mySig_float+noise, -32768,32767)        
        return noisySig.astype(np.int16)
    
    def timeShift(mySig, shift_lim=0.5):
        """Shifts the data in time (reintroduce ending values at the beginning."""
        _, sLen = mySig.shape
        shiftAmt = int(np.random.random() * shift_lim * sLen)
        return np.roll(mySig, shiftAmt, axis=0)
    
    @staticmethod
    def augment_fft(data, factor=1.2):
        """Augmente ou diminue certaines fréquences du signal"""
        
        fft_data = np.fft.fft(data, axis=0)
        fft_data = fft_data * factor  # Modification du spectre
        newData=np.real(np.fft.ifft(fft_data, axis=0))  # Retourne le signal temporel
        newData = np.clip(newData, -32768, 32767)
        return newData.astype(np.int16)

    # ----------------------------
    # Generate a Mel Spectrogram Excellent for Audio Data
    # ----------------------------
    @staticmethod
    def mel_spectro_gram(mySig, n_mels=64, n_fft=1024, hop_len=None, fs=50000):
        if hop_len is None:
            hop_len = n_fft // 4
            
        top_db = 80
        mySig=mySig.astype(np.float32)
        mySigTensor=torch.from_numpy(mySig.T)
        # spec has shape [channel, n_mels, time], where channel is mono, stereo etc
        spec = transforms.MelSpectrogram(fs, n_fft=n_fft, hop_length=hop_len, n_mels=n_mels)(mySigTensor)
        # Convert to decibels
        spec = transforms.AmplitudeToDB(top_db=top_db)(spec)
        spec = spec.unsqueeze(0) if spec.dim() == 2 else spec
        return (spec, hop_len)
    
    # ----------------------------
    # Generate a "classic" Spectrogram for Accelerometer Data
    # ----------------------------
    @staticmethod
    def spectro_gram(mySig, n_fft=1024, hop_len=None, fs=50000):
        if hop_len is None:
            hop_len = n_fft // 4
            
        mySig=mySig.astype(np.float32)
        mySigTensor=torch.from_numpy(mySig.T)
        # spec has shape [channel, n_mels, time], where channel is mono, stereo etc
        spec = transforms.Spectrogram(n_fft=n_fft, hop_length=hop_len, power=2)(mySigTensor)
        # Convert to decibels
        spec = transforms.AmplitudeToDB()(spec)
        spec = spec.unsqueeze(0) if spec.dim() == 2 else spec
        return (spec, hop_len)
    
    # ----------------------------
    # Visualize a Spectrogram
    # ----------------------------
    @staticmethod
    def show_spectrogram(spec, fs=50000, hop=256, title="Spectrogram", ylabel='Mel Frequency Bands', channels=['Sound', 'AccX', 'AccY', 'AccZ'], mel=False):
        nbChannels=spec.shape[0]
        fig, axs = plt.subplots(nbChannels, 1, figsize=(18, 12))
        if nbChannels==1:
            axs=[axs]
        timeAxis=np.arange(0, spec.shape[2]*hop, hop)/fs
        freqAxis = np.linspace(0, fs / 2, spec.shape[1])
        for i in range(nbChannels):
            if mel:
                img=axs[i].imshow(spec[i].detach().numpy(), aspect='auto', origin='lower', cmap='viridis', extent=[timeAxis[0], timeAxis[-1], 0, spec[i].shape[0]])
            else:
                img=axs[i].imshow(spec[i].detach().numpy(), aspect='auto', origin='lower', cmap='viridis', extent=[timeAxis[0], timeAxis[-1], freqAxis[0], freqAxis[-1]])
                
            axs[i].set_title(f"Channel {channels[i]}")
            axs[i].set_ylabel(ylabel)
        plt.xlabel('Time [s]')
        plt.suptitle(title)
        plt.colorbar(img, ax=axs, orientation='vertical')
        plt.show()
        
    # ----------------------------
    # Augment a Spectrogram
    # ----------------------------
    @staticmethod
    def spectro_Augment(spectro, maxMaskPercentage=0.1,nFreqMask=1, nTimeMasks=1):
        """Augmente ou diminue certaines fréquences du signal"""
        nChannel, nFreq, nSteps = spectro.shape
        mask_value=spectro.mean()
        aug_spec=spectro.clone()
        freqMaskParam=maxMaskPercentage*nFreq
        timeMaskParam=maxMaskPercentage*nSteps
        
        for ch in range(nChannel):
            channelSpec=aug_spec[ch]
            for _ in range(nFreqMask):
                channelSpec=transforms.FrequencyMasking(freqMaskParam)(channelSpec, mask_value)

            for _ in range(nTimeMasks):
                channelSpec=transforms.TimeMasking(timeMaskParam)(channelSpec, mask_value)
            aug_spec[ch]=channelSpec
            
        return aug_spec
        


In [8]:

from torch.utils.data import DataLoader, Dataset

#With TPU
from torch_xla.distributed.parallel_loader import MpDeviceLoader


#Creation of a custom Dataset for PyTorch

#df must contain Label and filepath only for one fold of training data. Folds will be dealt with externally to the DataLoader class.

#Datapath is the path to the folder containing all the folds (i.e. F:\Data_BachelorHES\DataSet_CNC\DataSetsFolds) 

class DataDS(Dataset):
    
    def __init__(self, df, datapath):
        self.df=df
        self.datapath=str(datapath)
        self.duration=5001.24
        self.fs=50000
        self.channel=4
        
    def __len__(self):
        return len(self.df)
    
    def __getitem__(self, idx):
        row=self.df.iloc[idx]
        dirFile=os.path.join(self.datapath,row['FilePath'])
        observationLoad=np.load(dirFile)
        labelId=row['LabelIDs']
        _, lenObs, obs=DataUtils.padTruncate(observationLoad, self.duration, self.fs)
        spec, _ = DataUtils.spectro_gram(obs, n_fft=1024, hop_len=None, fs=self.fs)
        augspec=DataUtils.spectro_Augment(spec, maxMaskPercentage=0.1,nFreqMask=2, nTimeMasks=2)
        return (augspec, labelId)

In [9]:
import torch.nn.functional as F
import torch.nn as nn
from torch.nn import init
#       Spectrogram   |  Shape  (nbChannel, nbFreq, nbTime)|  DataType 
#Augmented Spec shape |        torch.Size([4, 513, 977])   |  torch.FloatTensor

# ----------------------------
# Audio Classification Model
# ----------------------------
class ToolClassifier (nn.Module):
    # ----------------------------
    # Build the model architecture
    # ----------------------------
    def __init__(self):
        super().__init__()
        conv_layers = []

        # First Convolution Block with Relu and Batch Norm. Use Kaiming Initialization
        #torch.nn.Conv2d(in_channels, out_channels, kernel_size, stride=1, padding=0, dilation=1, groups=1, bias=True, padding_mode='zeros', device=None, dtype=None)
        #This convolution block 
        inChannels=4
        outChannels=16
        self.conv1 = nn.Conv2d(inChannels, outChannels, kernel_size=(5, 5), stride=(2, 2), padding=(2, 2))
        self.relu1 = nn.ReLU()
        self.bn1 = nn.BatchNorm2d(outChannels)
        init.kaiming_normal_(self.conv1.weight, a=0.1)
        self.conv1.bias.data.zero_()
        conv_layers += [self.conv1, self.relu1, self.bn1]

        # Second Convolution Block
        inChannels=16
        outChannels=32
        self.conv2 = nn.Conv2d(inChannels, outChannels, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
        self.relu2 = nn.ReLU()
        self.bn2 = nn.BatchNorm2d(outChannels)
        init.kaiming_normal_(self.conv2.weight, a=0.1)
        self.conv2.bias.data.zero_()
        conv_layers += [self.conv2, self.relu2, self.bn2]

        # Third Convolution Block
        inChannels=32
        outChannels=64
        self.conv3 = nn.Conv2d(inChannels, outChannels, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
        self.relu3 = nn.ReLU()
        self.bn3 = nn.BatchNorm2d(outChannels)
        init.kaiming_normal_(self.conv3.weight, a=0.1)
        self.conv3.bias.data.zero_()
        conv_layers += [self.conv3, self.relu3, self.bn3]

        # Fourth Convolution Block
        inChannels=64
        outChannels=128
        self.conv4 = nn.Conv2d(inChannels, outChannels, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
        self.relu4 = nn.ReLU()
        self.bn4 = nn.BatchNorm2d(outChannels)
        init.kaiming_normal_(self.conv4.weight, a=0.1)
        self.conv4.bias.data.zero_()
        conv_layers += [self.conv4, self.relu4, self.bn4]

        # Linear Classifier
        self.ap = nn.AdaptiveAvgPool2d(output_size=1)
        self.lin = nn.Linear(in_features=outChannels, out_features=11)

        # Wrap the Convolutional Blocks
        self.conv = nn.Sequential(*conv_layers)
 
    # ----------------------------
    # Forward pass computations
    # ----------------------------
    def forward(self, x):
        # Run the convolutional blocks
        x = self.conv(x)

        # Adaptive pool and flatten for input to linear layer
        x = self.ap(x)
        x = x.view(x.shape[0], -1)

        # Linear layer
        x = self.lin(x)

        # Final output
        return x



In [10]:
from sklearn.metrics import confusion_matrix, classification_report

# ----------------------------
# Inference
# ----------------------------
def inference (model, test_dl, device, trainMean, trainStd ):
    model.eval() #Put the model in eval/testing mode
    allPreds=[]
    allLabels=[]
    correct_prediction = 0
    total_prediction = 0
    
    # Disable gradient updates
    with torch.no_grad():
        for data in test_dl:
            # Get the input features and target labels, and put them on the GPU
            inputs, labels = data[0].to(device), data[1].to(device)
            
            # Normalize the inputs
            inputs = (inputs - trainMean.view(1,-1,1,1)) / trainStd.view(1,-1,1,1)
            
            # Get predictions
            outputs = model(inputs)
            
            # Get the predicted class with the highest score
            _, prediction = torch.max(outputs,1)
            # Count of predictions that matched the target label
            correct_prediction += (prediction == labels).sum().item()
            total_prediction += prediction.shape[0]
            allPreds.append(prediction.cpu().numpy())
            allLabels.append(labels.cpu().numpy())

    #Concatenate all results over the entire test set
    allPreds = np.concatenate(allPreds)
    allLabels = np.concatenate(allLabels)
    
    acc = correct_prediction/total_prediction
    xm.master_print(f'On inference : Accuracy: {acc:.2f}, Total items: {total_prediction}')

    # Compute Confusion Matrix
    confMatrix = confusion_matrix(allLabels, allPreds)


     # Comput Classification Report (precision, recall, f1-score, support)
    classReport = classification_report(allLabels, allPreds,output_dict=True, digits=4, zero_division=0)


    return confMatrix, classReport
    


In [11]:
# ----------------------------
# Training Loop
# ----------------------------
def normalize_ds(dataloader, device):
    #Compute mean and standard deviation accross all train data
    # results are mean and std for every spectrogram accross all channels (here mean and std have size (4,513,977))
    sumInp=None
    sumSqDiff=None
    totObs=0
    
    #iterate over data to calculate global data mean
    for i, data in enumerate(dataloader): 
        
        inputs =data[0].to(device)
        if sumInp is None:
            sumInp=torch.zeros(inputs.size(1),inputs.size(2),inputs.size(3), device=device)
            sumSqDiff=torch.zeros(inputs.size(1),inputs.size(2),inputs.size(3), device=device)
            
        sumInp+=inputs.sum(dim=0)
        totObs+=inputs.size(0)
        del inputs  # ✅ Libère la mémoire
        xm.mark_step()  # ✅ Assure la libération sur TPU
        
    totMean=sumInp/totObs
    
    for i, data in enumerate(dataloader):
        inputs =data[0].to(device)
        sumSqDiff+=((inputs-totMean)**2).sum(dim=0)

        del inputs  # ✅ Libère la mémoire
        xm.mark_step()  # ✅ Assure la libération sur TPU
        
    totVar=sumSqDiff/totObs
    totStd=torch.sqrt(totVar+1e-8)
    return totMean, totStd


def training(model, device, train_dl, num_epochs):
    #store_results
    accuracies=[]
    losses=[]
    #Put the model in training mode 
    model.train()
    # Loss Function, Optimizer and Scheduler
    criterion = nn.CrossEntropyLoss()
    optimizer = torch.optim.Adam(model.parameters(),lr=0.001)
    scheduler = torch.optim.lr_scheduler.OneCycleLR(optimizer, max_lr=0.001,
                                                steps_per_epoch=int(len(train_dl)),
                                                epochs=num_epochs,
                                                anneal_strategy='linear')

    trainMean, trainStd=normalize_ds(train_dl, device)
    
    # Repeat for each epoch
    for epoch in range(num_epochs):
        running_loss = 0.0
        correct_prediction = 0
        total_prediction = 0
        
        # Repeat for each batch in the training set
        for i, data in enumerate(train_dl):
            # Get the input features and target labels, and put them on the GPU
            inputs, labels = data[0].to(device), data[1].to(device)
        
            # Normalize the inputs
            inputs = (inputs - trainMean.view(1,*trainMean.shape)) / trainStd.view(1,*trainStd.shape)
        
            # Zero the parameter gradients
            optimizer.zero_grad()
        
            # forward + backward + optimize
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            xm.optimizer_step(optimizer, barrier=True)
            scheduler.step()
        
            # Keep stats for Loss and Accuracy
            running_loss += loss.item()
        
            # Get the predicted class with the highest score
            _, prediction = torch.max(outputs,1)
            # Count of predictions that matched the target label
            correct_prediction += (prediction == labels).sum().item()
            total_prediction += prediction.shape[0]
        
            #if i % 10 == 0:    # print every 10 mini-batches
            #    print('[%d, %5d] loss: %.3f' % (epoch + 1, i + 1, running_loss / 10))
            
        # Print stats at the end of the epoch
        num_batches = len(train_dl)
        avg_loss = running_loss / num_batches
        acc = correct_prediction/total_prediction
        accuracies.append(acc)
        losses.append(avg_loss)
        xm.master_print(f'Epoch: {epoch}, Loss: {avg_loss:.2f}, Accuracy: {acc:.2f}')
    
    xm.master_print('Finished Training')
    return accuracies, losses, trainMean, trainStd         
    

In [13]:
%who

DataDS	 DataLoader	 DataUtils	 Dataset	 F	 MpDeviceLoader	 ToolClassifier	 classification_report	 confusion_matrix	 
csv	 dirname	 filenames	 inference	 init	 math	 met	 myGlobalDs	 myMeta	 
nn	 normalize_ds	 np	 os	 pd	 pl	 plt	 random	 test_utils	 
torch	 torch_xla	 torchaudio	 training	 transforms	 warnings	 xm	 xmp	 xu	 


## Learning curve with respect to number of epochs

In [19]:
import os
import gc
import torch
import torch.nn as nn
import torch.optim as optim
import torch_xla.core.xla_model as xm
import torch_xla.distributed.parallel_loader as pl
import torch_xla.distributed.xla_multiprocessing as xmp
from torch.utils.data import DataLoader, DistributedSampler

In [20]:


def training_one_epoch(model, optimizer, criterion, scheduler, device, train_dl, trainMean, trainStd, epochNb):
        #Put the model in training mode 
    model.train()
    running_loss = 0.0
    correct_prediction = 0
    total_prediction = 0
    
    # Repeat for each batch in the training set
    for i, data in enumerate(train_dl):
        # Get the input features and target labels, and put them on the GPU
        inputs, labels = data[0].to(device), data[1].to(device)
    
        # Normalize the inputs
        inputs = (inputs - trainMean.view(1,*trainMean.shape)) / trainStd.view(1,*trainStd.shape)
    
        # Zero the parameter gradients
        optimizer.zero_grad()
    
        # forward + backward + optimize
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        

        #Use ofd TPU + Scheduler
        xm.optimizer_step(optimizer, barrier=True)
        xm.mark_step()  # Synchronisation du TPU
        scheduler.step()
    
        # Keep stats for Loss and Accuracy
        running_loss += loss.item()
    
        # Get the predicted class with the highest score
        _, prediction = torch.max(outputs,1)
        # Count of predictions that matched the target label
        correct_prediction += (prediction == labels).sum().item()
        total_prediction += prediction.shape[0]
            
    # Print stats at the end of the epoch for the training stage
    num_batches = len(train_dl)
    avg_loss = running_loss / num_batches
    acc = correct_prediction/total_prediction
    xm.master_print(f'Epoch Training: {epochNb}, Loss: {avg_loss:.4f}, Accuracy: {acc:.4f}')
    return avg_loss, acc

def testing_one_epoch(model, criterion, device, test_dl, trainMean, trainStd, epochNb):
    #Evaluate model for each epoch
    model.eval()
    test_loss = 0.0
    correct=0
    total=0
    
    with torch.no_grad():     
        for inputs, labels in test_dl:
            inputs, labels = inputs.to(device), labels.to(device)
            
            # Normalize the inputs
            inputs = (inputs - trainMean.view(1,*trainMean.shape)) / trainStd.view(1,*trainStd.shape)

            #Get prediciitons
            outputs = model(inputs)
            _, prediction = torch.max(outputs, 1)
            
            loss = criterion(outputs, labels)
            test_loss += loss.item()

            correct += (prediction == labels).sum().item()
            total += prediction.shape[0]

                           
    test_acc = correct/total
    avg_test_loss=test_loss / len(test_dl)
    xm.master_print(f'Epoch Test: {epochNb}, Loss: {avg_test_loss:.4f}, Accuracy: {test_acc:.4f}')
    return avg_test_loss, test_acc

In [21]:
def process_one_fold(run, device, results_queue, myGlobalDs, mainDir, nbEpochs):
    

    # 📌 Chargement des données
    myTrain = myGlobalDs[(myGlobalDs['Fold'] == run) & (myGlobalDs['Type'] == 'Train')].copy()
    myTrain = myTrain.drop(columns=['Fold', 'Type', 'nb', 'condLabel'])
    trainFoldi = DataDS(myTrain, mainDir)

    myTest = myGlobalDs[(myGlobalDs['Fold'] == run) & (myGlobalDs['Type'] == 'Test')].copy()
    myTest = myTest.drop(columns=['Fold', 'Type', 'nb', 'condLabel'])
    testFoldi = DataDS(myTest, mainDir)

    # Distribute datasets on 8 TPU Cores
    train_sampler = DistributedSampler(trainFoldi, num_replicas=xm.xrt_world_size(), rank=xm.get_ordinal(), shuffle=True)
    test_sampler = DistributedSampler(testFoldi, num_replicas=xm.xrt_world_size(), rank=xm.get_ordinal(), shuffle=False)

    # Create the DataLoaders
    train_dataloader = DataLoader(trainFoldi, batch_size=8, sampler=train_sampler)
    test_dataloader = DataLoader(testFoldi, batch_size=8, sampler=test_sampler)

    # Create the model to be trained
    myModel = ToolClassifier().to(device)
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(myModel.parameters(), lr=0.001)
    scheduler = torch.optim.lr_scheduler.OneCycleLR(optimizer, max_lr=0.001, steps_per_epoch=len(train_dataloader), epochs=nbEpochs, anneal_strategy='linear')

    # Get global train mean an dstd for normalization
    trainMean, trainStd = normalize_ds(train_dataloader, device)
    
    # List to store results
    train_losses, test_losses, train_accuracies, test_accuracies = [], [], [], []

    # train -> test -> train loop for each epoch
    #training_one_epoch(model, optimizer, criterion, scheduler, device, train_dl, trainMean, trainStd, epochNb)
    #testing_one_epoch(model, criterion, device, test_dl, trainMean, trainStd, epochNb)
    for epoch in range(nbEpochs):
        train_loss, train_acc = training_one_epoch(myModel, optimizer, criterion, scheduler, device, train_dataloader, trainMean, trainStd, epoch)
        test_loss, test_acc = testing_one_epoch(myModel, criterion, device, test_dataloader, trainMean, trainStd, epoch)

        train_losses.append(train_loss)
        test_losses.append(test_loss)
        train_accuracies.append(train_acc)
        test_accuracies.append(test_acc)

        torch.cuda.empty_cache()  # Libère la mémoire GPU si applicable
        gc.collect()  # Libère la mémoire du CPU
        xm.mark_step()  # Libère la mémoire TPU

        # A TPU sync at the end of an epoch 
        xm.rendezvous("epoch_sync")

    xm.master_print("Finished Training !")
    # Send results to main process
    results_queue.put((train_losses, test_losses, train_accuracies, test_accuracies))

    # Cleanup of TPU memory
    del myModel
    gc.collect()
    xm.mark_step()  
    xm.rendezvous("cleanup")

In [22]:
def _mp_fn(index, results_queue, nbFold, myGlobalDS, mainDir, num_epochs):
    device = xm.xla_device()
    for run in range(nbFold):
        xm.master_print(f"\n Run {run+1}/{nbFold} sur TPU Core {xm.get_ordinal()}...\n")
        process_one_fold(run, device, results_queue, myGlobalDS, mainDir, num_epochs)

In [23]:
#from multiprocessing import Queue
import torch.multiprocessing as mp


import time
mainDir='/kaggle/input/mynumpyfiles/NumpyDS'
nbFold=5
num_epochs=60
manager = mp.Manager()  # ✅ Crée un manager pour gérer des objets entre processus
results_queue = manager.Queue()  # ✅ Création de la queue compatible avec `xmp.spawn()`

xmp.spawn(_mp_fn, args=(results_queue, nbFold, myGlobalDs, mainDir, num_epochs), nprocs=xm.xrt_world_size(), start_method='fork')

# 📌 Collecte des résultats
train_losses, test_losses, train_accuracies, test_accuracies = [], [], [], []
time.sleep(5) 
while not results_queue.empty():
    t_loss, te_loss, t_acc, te_acc = results_queue.get()
    train_losses.append(t_loss)
    test_losses.append(te_loss)
    train_accuracies.append(t_acc)
    test_accuracies.append(te_acc)

# 📌 Sauvegarde des résultats
np.save("/kaggle/working/epoch_train_losses.npy", np.array(train_losses))
np.save("/kaggle/working/epoch_test_losses.npy", np.array(test_losses))
np.save("/kaggle/working/epoch_train_accuracies.npy", np.array(train_accuracies))
np.save("/kaggle/working/epoch_test_accuracies.npy", np.array(test_accuracies))

E0000 00:00:1740430624.185606      74 common_lib.cc:621] Could not set metric server port: INVALID_ARGUMENT: Could not find SliceBuilder port 8471 in any of the 0 ports provided in `tpu_process_addresses`="local"
=== Source Location Trace: ===
learning/45eac/tfrc/runtime/common_lib.cc:239



 Run 1/5 sur TPU Core 0...


Epoch Training: 0, Loss: 2.1754, Accuracy: 0.2851


Epoch Test: 0, Loss: 2.1399, Accuracy: 0.3433


Epoch Training: 1, Loss: 1.7847, Accuracy: 0.4228


...

Epoch Training: 58, Loss: 0.0026, Accuracy: 1.0000


Epoch Test: 58, Loss: 0.1426, Accuracy: 0.9728


Epoch Training: 59, Loss: 0.0038, Accuracy: 0.9995


Epoch Test: 59, Loss: 0.1558, Accuracy: 0.9755


Finished Training !
